In [17]:
### Sagar Raut PRN : 240340128027
### Ameya Panshikar PRN : 240340128002

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier as XGBoost
from lightgbm import LGBMClassifier as LGBoost
from catboost import CatBoostClassifier as CATboost

In [3]:
# Read training and testing data
train_data = pd.read_csv(r"D:\PML\Assignment\Santander Customer Satisfaction(Assign2)\Santander Customer Satisfaction(Assign2)\train.csv")
test_data = pd.read_csv(r"D:\PML\Assignment\Santander Customer Satisfaction(Assign2)\Santander Customer Satisfaction(Assign2)\test.csv")


In [4]:
# Select features (excluding ID and target)
features = train_data.columns[1:-1]  # Exclude 'ID' and 'TARGET' columns

In [5]:
# Preprocess training data
scaler = StandardScaler().set_output(transform='pandas')
scaler.fit(train_data[features])
train_scaled = scaler.transform(train_data[features])

In [6]:
# Feature dimensionality reduction with PCA
pca = PCA().set_output(transform='pandas')
principal_components = pca.fit_transform(train_scaled)

In [13]:
# Determine number of components for 90% explained variance
explained_variance = pd.Series(np.cumsum(pca.explained_variance_ratio_) * 100)
explained_variance
num_components =explained_variance[explained_variance.values > 90].index[0]
num_components

74

In [22]:
# Machine learning pipeline 
pipe = Pipeline([
    ('std_scaler', StandardScaler()),
    ('PCA', PCA(n_components=num_components)),
    ('RFC', RandomForestClassifier())
])

In [25]:
kfold = StratifiedKFold(n_splits=5, shuffle=True,
                        random_state=24)
params = {}
gcv = GridSearchCV(pipe, param_grid=params, cv=kfold ,
                   scoring ='roc_auc')
gcv.fit(train_data[features], train_data['TARGET'])
print(gcv.best_params_)
print(gcv.best_score_)

{}
0.7271616927994574


In [26]:
# Define additional models for comparison (LGBM, XGBoost, CatBoost)
pipe_lgbm = Pipeline([
    ('std_scaler', StandardScaler()),
    ('PCA', PCA()),
    ('LGBM', LGBoost())
])
pipe_xgbm = Pipeline([
    ('std_scaler', StandardScaler()),
    ('PCA', PCA()),
    ('XGBM', XGBoost())
])
pipe_cat = Pipeline([
    ('std_scaler', StandardScaler()),
    ('PCA', PCA()),
    ('CAT', CATboost())
])


In [20]:
# Train and predict with LGBM, XGBoost, CatBoost and pipe
pipe_lgbm.fit(train_data[features], train_data['TARGET'])
y_pred_lgbm = pipe_lgbm.predict(test_data[features])
y_pred_prob_lgbm = pipe_lgbm.predict_proba(test_data[features])[:, 1]

pipe_xgbm.fit(train_data[features], train_data['TARGET'])
y_pred_xgbm = pipe_xgbm.predict(test_data[features])
y_pred_prob_xgbm = pipe_xgbm.predict_proba(test_data[features])[:, 1]

pipe_cat.fit(train_data[features], train_data['TARGET'])
y_pred_cat = pipe_cat.predict(test_data[features])
y_pred_prob_cat = pipe_cat.predict_proba(test_data[features])[:, 1]

pipe.fit(train_data[features], train_data['TARGET'])
y_pred_pipe = pipe.predict(test_data[features])
y_pred_prob_pipe = pipe.predict_proba(test_data[features])[:, 1]

[LightGBM] [Info] Number of positive: 3008, number of negative: 73012
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.089347 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 94095
[LightGBM] [Info] Number of data points in the train set: 76020, number of used features: 369
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.039569 -> initscore=-3.189348
[LightGBM] [Info] Start training from score -3.189348
Learning rate set to 0.065477
0:	learn: 0.5937660	total: 239ms	remaining: 3m 58s
1:	learn: 0.5176352	total: 298ms	remaining: 2m 28s
2:	learn: 0.4524869	total: 358ms	remaining: 1m 59s
3:	learn: 0.3969111	total: 416ms	remaining: 1m 43s
4:	learn: 0.3532953	total: 487ms	remaining: 1m 36s
5:	learn: 0.3178726	total: 552ms	remaining: 1m 31s
6:	learn: 0.2881709	total: 613ms	remaining: 1m 26s
7:	learn: 0.2667662	total: 664ms	remaining: 1m 22s
8:	learn: 0.2480402	total: 719ms	remaining: 1m 19s
9:	learn: 0.2321

154:	learn: 0.1276428	total: 8.86s	remaining: 48.3s
155:	learn: 0.1275843	total: 8.92s	remaining: 48.2s
156:	learn: 0.1275310	total: 8.98s	remaining: 48.2s
157:	learn: 0.1274653	total: 9.04s	remaining: 48.2s
158:	learn: 0.1273984	total: 9.09s	remaining: 48.1s
159:	learn: 0.1273265	total: 9.14s	remaining: 48s
160:	learn: 0.1272463	total: 9.19s	remaining: 47.9s
161:	learn: 0.1271515	total: 9.25s	remaining: 47.8s
162:	learn: 0.1270645	total: 9.3s	remaining: 47.8s
163:	learn: 0.1270073	total: 9.36s	remaining: 47.7s
164:	learn: 0.1269219	total: 9.41s	remaining: 47.6s
165:	learn: 0.1268485	total: 9.46s	remaining: 47.6s
166:	learn: 0.1267573	total: 9.52s	remaining: 47.5s
167:	learn: 0.1267044	total: 9.57s	remaining: 47.4s
168:	learn: 0.1266408	total: 9.62s	remaining: 47.3s
169:	learn: 0.1265799	total: 9.67s	remaining: 47.2s
170:	learn: 0.1265109	total: 9.73s	remaining: 47.2s
171:	learn: 0.1264552	total: 9.79s	remaining: 47.1s
172:	learn: 0.1263960	total: 9.84s	remaining: 47s
173:	learn: 0.126

315:	learn: 0.1168914	total: 17.5s	remaining: 37.9s
316:	learn: 0.1168344	total: 17.6s	remaining: 37.9s
317:	learn: 0.1167966	total: 17.6s	remaining: 37.8s
318:	learn: 0.1167375	total: 17.7s	remaining: 37.7s
319:	learn: 0.1166636	total: 17.7s	remaining: 37.7s
320:	learn: 0.1166102	total: 17.8s	remaining: 37.6s
321:	learn: 0.1165281	total: 17.8s	remaining: 37.5s
322:	learn: 0.1164635	total: 17.9s	remaining: 37.5s
323:	learn: 0.1163973	total: 17.9s	remaining: 37.4s
324:	learn: 0.1163479	total: 18s	remaining: 37.4s
325:	learn: 0.1162630	total: 18.1s	remaining: 37.3s
326:	learn: 0.1161824	total: 18.1s	remaining: 37.3s
327:	learn: 0.1161089	total: 18.2s	remaining: 37.2s
328:	learn: 0.1160639	total: 18.2s	remaining: 37.2s
329:	learn: 0.1160222	total: 18.3s	remaining: 37.1s
330:	learn: 0.1159545	total: 18.3s	remaining: 37.1s
331:	learn: 0.1159002	total: 18.4s	remaining: 37s
332:	learn: 0.1158187	total: 18.4s	remaining: 36.9s
333:	learn: 0.1157784	total: 18.5s	remaining: 36.9s
334:	learn: 0.11

474:	learn: 0.1078587	total: 26.3s	remaining: 29s
475:	learn: 0.1078126	total: 26.3s	remaining: 29s
476:	learn: 0.1077688	total: 26.4s	remaining: 28.9s
477:	learn: 0.1077196	total: 26.4s	remaining: 28.9s
478:	learn: 0.1076793	total: 26.5s	remaining: 28.8s
479:	learn: 0.1076020	total: 26.5s	remaining: 28.8s
480:	learn: 0.1075389	total: 26.6s	remaining: 28.7s
481:	learn: 0.1074749	total: 26.7s	remaining: 28.6s
482:	learn: 0.1074029	total: 26.7s	remaining: 28.6s
483:	learn: 0.1073929	total: 26.8s	remaining: 28.5s
484:	learn: 0.1073456	total: 26.8s	remaining: 28.5s
485:	learn: 0.1072896	total: 26.9s	remaining: 28.4s
486:	learn: 0.1072290	total: 26.9s	remaining: 28.4s
487:	learn: 0.1071835	total: 27s	remaining: 28.3s
488:	learn: 0.1071285	total: 27s	remaining: 28.2s
489:	learn: 0.1070916	total: 27.1s	remaining: 28.2s
490:	learn: 0.1070504	total: 27.1s	remaining: 28.1s
491:	learn: 0.1070195	total: 27.2s	remaining: 28.1s
492:	learn: 0.1069645	total: 27.2s	remaining: 28s
493:	learn: 0.1069152	

636:	learn: 0.1001437	total: 35.3s	remaining: 20.1s
637:	learn: 0.1001001	total: 35.4s	remaining: 20.1s
638:	learn: 0.1000607	total: 35.5s	remaining: 20s
639:	learn: 0.1000136	total: 35.5s	remaining: 20s
640:	learn: 0.0999623	total: 35.6s	remaining: 19.9s
641:	learn: 0.0999266	total: 35.6s	remaining: 19.9s
642:	learn: 0.0998843	total: 35.7s	remaining: 19.8s
643:	learn: 0.0998520	total: 35.7s	remaining: 19.8s
644:	learn: 0.0998177	total: 35.8s	remaining: 19.7s
645:	learn: 0.0997656	total: 35.8s	remaining: 19.6s
646:	learn: 0.0997121	total: 35.9s	remaining: 19.6s
647:	learn: 0.0996695	total: 36s	remaining: 19.5s
648:	learn: 0.0996197	total: 36s	remaining: 19.5s
649:	learn: 0.0995710	total: 36.1s	remaining: 19.4s
650:	learn: 0.0995414	total: 36.1s	remaining: 19.4s
651:	learn: 0.0994823	total: 36.2s	remaining: 19.3s
652:	learn: 0.0994431	total: 36.2s	remaining: 19.3s
653:	learn: 0.0993912	total: 36.3s	remaining: 19.2s
654:	learn: 0.0993544	total: 36.3s	remaining: 19.1s
655:	learn: 0.099294

798:	learn: 0.0937325	total: 44.3s	remaining: 11.1s
799:	learn: 0.0937159	total: 44.4s	remaining: 11.1s
800:	learn: 0.0936844	total: 44.4s	remaining: 11s
801:	learn: 0.0936558	total: 44.5s	remaining: 11s
802:	learn: 0.0936145	total: 44.5s	remaining: 10.9s
803:	learn: 0.0935737	total: 44.6s	remaining: 10.9s
804:	learn: 0.0935280	total: 44.7s	remaining: 10.8s
805:	learn: 0.0934881	total: 44.7s	remaining: 10.8s
806:	learn: 0.0934587	total: 44.8s	remaining: 10.7s
807:	learn: 0.0934056	total: 44.8s	remaining: 10.7s
808:	learn: 0.0933746	total: 44.9s	remaining: 10.6s
809:	learn: 0.0933438	total: 44.9s	remaining: 10.5s
810:	learn: 0.0932996	total: 45s	remaining: 10.5s
811:	learn: 0.0932695	total: 45.1s	remaining: 10.4s
812:	learn: 0.0932377	total: 45.1s	remaining: 10.4s
813:	learn: 0.0931895	total: 45.2s	remaining: 10.3s
814:	learn: 0.0931557	total: 45.2s	remaining: 10.3s
815:	learn: 0.0930994	total: 45.3s	remaining: 10.2s
816:	learn: 0.0930785	total: 45.3s	remaining: 10.2s
817:	learn: 0.0930

960:	learn: 0.0883770	total: 53.5s	remaining: 2.17s
961:	learn: 0.0883503	total: 53.6s	remaining: 2.12s
962:	learn: 0.0883269	total: 53.6s	remaining: 2.06s
963:	learn: 0.0883040	total: 53.7s	remaining: 2s
964:	learn: 0.0882628	total: 53.7s	remaining: 1.95s
965:	learn: 0.0882276	total: 53.8s	remaining: 1.89s
966:	learn: 0.0882063	total: 53.8s	remaining: 1.84s
967:	learn: 0.0881767	total: 53.9s	remaining: 1.78s
968:	learn: 0.0881536	total: 54s	remaining: 1.73s
969:	learn: 0.0881263	total: 54s	remaining: 1.67s
970:	learn: 0.0880932	total: 54.1s	remaining: 1.61s
971:	learn: 0.0880576	total: 54.1s	remaining: 1.56s
972:	learn: 0.0880303	total: 54.2s	remaining: 1.5s
973:	learn: 0.0879917	total: 54.3s	remaining: 1.45s
974:	learn: 0.0879433	total: 54.3s	remaining: 1.39s
975:	learn: 0.0879078	total: 54.4s	remaining: 1.34s
976:	learn: 0.0878640	total: 54.4s	remaining: 1.28s
977:	learn: 0.0878405	total: 54.5s	remaining: 1.23s
978:	learn: 0.0878071	total: 54.6s	remaining: 1.17s
979:	learn: 0.087774

In [28]:
# Prepare and save submission files for each model
submit_lgbm = pd.DataFrame({'ID': test_data['ID'], 'TARGET': y_pred_prob_lgbm})
submit_lgbm.to_csv('sbt_santander_lgbm.csv', index=False)

submit_xgbm = pd.DataFrame({'ID': test_data['ID'], 'TARGET': y_pred_prob_xgbm})
submit_xgbm.to_csv('sbt_santander_xgbm.csv', index=False)

submit_cat = pd.DataFrame({'ID': test_data['ID'], 'TARGET': y_pred_prob_cat})
submit_cat.to_csv('sbt_santander_cat.csv', index=False)

submit_pipe = pd.DataFrame({'ID': test_data['ID'], 'TARGET': y_pred_prob_pipe})
submit_pipe.to_csv('sbt_santander_rf.csv', index=False)